# SWOP script 6
## Visualize results of mixed-effects modelling

Analysis script associated with the manuscript: ***Native word order processing is not uniform: An ERP-study of verb-second word order***, by Susan Sayehli, Marianne Gullberg, Aaron Newman, and Annika Andersson. Currently under review with *Frontiers in Psychology - Language Sciences*, manuscript 668276.

This notebook reads tables generated by the scripts *SWOP 5**, which compute linear mixed-effects models on the data in various time windows. This script plots the model-estimated ERP amplitudes for various conditions/contrasts and ROIS, along with model-derived 95% confidence intervals. It also plots these as effect sizes. The results are identical whether model estimates or effecti sizes are on the y axis; the difference is merely in the units.

---
Copyright 2016-21  [Aaron J Newman](https://github.com/aaronjnewman), [NeuroCognitive Imaging Lab](http://ncil.science), [Dalhousie University](https://dal.ca)

Released under the [The 3-Clause BSD License](https://opensource.org/licenses/BSD-3-Clause)

---

In [1]:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd

## Plot parameters

In [2]:
# Change this to 300 for generating hi rez (poster/paper) images. Use 72 for on-screen viewing.
dpi = 300
matplotlib.rcParams['figure.dpi'] = dpi  
# plt.style.use('seaborn-paper')

eff_size_ylim = [-.35, .46]
mod_est_ylim = [-1.5, 2.25]
by_roi_figsize = [6, 6]
by_adv_figsize = [6, 10]

## Time windows of interest

In [3]:
time_wins = ['100-300', '300-500', '500-700', '700-900', '900-1000']

## Conditions of Interest

In [4]:
adverbs = ['kanske', 'hemma', 'idag']
colors = ['blue', 'green', 'orangered']
color_map = dict(zip(adverbs, colors))

## ROIS

In [5]:
roi_map = {'L_Ant':'L Ant',
           'M_Ant':'M Ant',
           'R_Ant':'R Ant',
           'L_Cent':'L Cent', 
           'M_Cent':'M Cent', 
           'R_Cent':'R Cent',
           'L_Post':'L Post',
           'M_Post':'M Post',
           'R_Post':'R Post',
          }

roi_order = ['L Ant',  'M Ant',  'R Ant',
             'L Cent', 'M Cent', 'R Cent',
             'L Post', 'M Post', 'R Post'
            ]

roi_levels = ['Ant', 'Cent', 'Post']

## Generate plots

The warnings generated can safely be ignored

In [6]:
lme_path = '../results/'

for tw in time_wins:
    ## Read in LME output from R
    df = pd.read_csv(lme_path + 'contrast_table_' + tw + '.csv', index_col=[0])
    
    ### Compute CIs for model estimates
    df['lower.CI'] = df['estimate'] - 1.96 * df['SE.ctr']
    df['upper.CI'] = df['estimate'] + 1.96 * df['SE.ctr']
    df['CI.range'] = df['upper.CI'] - df['lower.CI']

    ### Give ROIs nice names for plots
    df = df.replace({'ROI':roi_map})

    #########
    # PLOT 1
    ## Point plots of *effect sizes* of V3-V2 contrast, arranged by ROI

    fig, axs = plt.subplots(3, 3, figsize=by_roi_figsize)    

    for idx, ax in enumerate(axs.reshape(-1)):
        roi = roi_order[idx]

        ax.axhline(color='darkgray', linestyle='--')

       ## Plot data points
        for x, y, c in zip(df[(df['ROI']==roi)]['Adverb'], 
                           df[df['ROI']==roi]['effect.size'],
                           colors
                       ):
            ax.plot(x, y, marker='o', color=c)
    
        ## Plot CIs
        for lower, upper, y, c in zip(df[df['ROI']==roi]['lower.CL'], 
                                      df[df['ROI']==roi]['upper.CL'], 
                                      range(len(df[df['ROI']==roi])),
                                      colors
                                  ):
            ax.plot((y,y), (lower,upper), '-', color=c) 

        ax.set_ylim(eff_size_ylim)

        if idx == 0:
            ax.set_ylabel('V3-V2 Effect Size (Cohen\'s d)')
        elif idx % 3 != 0:
            ax.set_yticks([])
            ax.spines['left'].set_visible(False)

        ax.set_xlabel('')
        ax.tick_params(axis='x', which='both', length=0)
        if idx < 6:
            ax.set_xticks([])    

        ax.set_title(roi)

        ax.spines['right'].set_visible(False)
        ax.spines['bottom'].set_visible(False)
        ax.spines['top'].set_visible(False)
        

    plt.tight_layout()
    fig.savefig(lme_path + 'figures/' + str(dpi) + ' dpi/lme_EffSize_' + tw + '_' + str(dpi) + 'dpi.png');
    fig.savefig(lme_path + 'figures/svg/lme_EffSize_' + tw + '.svg');

    plt.close()

    #########
    # PLOT 2
    ## Point plots of *effect sizes* for each V3-V2 contrast, grouped by adverb (condition)
    
    fig, axs = plt.subplots(3, 1, figsize=by_adv_figsize)
    for idx, ax in enumerate(axs.reshape(-1)):
        contr = adverbs[idx]

        for label in (ax.get_xticklabels() + ax.get_yticklabels()):
            label.set_fontsize(11)

        ax.plot(df[df['Adverb']==contr]['ROI'], df[df['Adverb']==contr]['effect.size'],
                linestyle='None', marker='o', color=color_map[contr])

        for lower, upper, y in zip(df[df['Adverb']==contr]['lower.CL'], 
                                   df[df['Adverb']==contr]['upper.CL'], 
                                   range(len(df))):
            ax.plot((y,y), (lower,upper),'-', color=color_map[contr])

        ax.axhline(0, color='darkgray', linestyle='--')
        ax.axvline(2.5, color='lightgray')
        ax.axvline(5.5, color='lightgray')

        if idx == 0:
            ax.set_ylabel('V3-V2 Effect Size (Cohen\'s d)')
        else:
             ax.set_ylabel('')

        if idx == 2:
            ax.set_xlabel('ROI')
            ax.set_xticklabels(roi_order, rotation=0)
        else:
            ax.set_xticks([])

        ax.set_title(contr)

        ax.set_ylim(eff_size_ylim)

        plt.tick_params(
            axis='x',          # changes apply to the x-axis
            which='both',      # both major and minor ticks are affected
            bottom=False,      # ticks along the bottom edge 
            top=False,         # ticks along the top edge 
            labelbottom=True) # labels along the bottom edge 

        ax.spines['right'].set_visible(False)
        ax.spines['bottom'].set_visible(False)
        ax.spines['top'].set_visible(False)

    plt.tight_layout()
    fig.savefig(lme_path + 'figures/' + str(dpi) + ' dpi/lme_EffSize_byAdverb' + '_' + tw + '_' + str(dpi) + 'dpi.png');
    fig.savefig(lme_path + 'figures/svg/lme_EffSize_byAdverb' + '_' + tw + '.svg');
    plt.close()
    
    #########
    # PLOT 3
    ## Point plots f effect size, columns are adverbs and rows are ant/mid/cent
    fig, axs = plt.subplots(3, 3, figsize=by_roi_figsize)    

    for idx, ax in enumerate(axs.reshape(-1)):
        adverb = adverbs[idx % 3]
        rois = [r for r in roi_order if roi_levels[idx // 3] in r]   

        ax.plot(rois, 
                df[(df['ROI'].isin(rois)) & (df['Adverb']==adverb)]['effect.size'], 
                linestyle='None', marker='o', color=color_map[adverb]
               )

        for lower, upper, y in zip(df[df['ROI'].isin(rois) & (df['Adverb']==adverb)]['lower.CL'], 
                                      df[df['ROI'].isin(rois) & (df['Adverb']==adverb)]['upper.CL'], 
                                      range(len(df[df['ROI'].isin(rois) & (df['Adverb']==adverb)])),
                                  ):
            ax.plot((y,y), (lower,upper), '-', color=color_map[adverb]) 

        if idx < 3:
            ax.set_title(adverb)

        if idx == 0:
            ax.set_ylabel('V3-V2 Effect Size (Cohen\'s d)')
        elif idx % 3 != 0:
            ax.set_yticks([])
            ax.spines['left'].set_visible(False)

        ax.set_ylim(eff_size_ylim)

        ax.set_xticklabels(rois, rotation=0)
        ax.tick_params(axis='x', which='both', length=0)

        ax.spines['right'].set_visible(False)
        ax.spines['bottom'].set_visible(False)
        ax.spines['top'].set_visible(False)

        ax.axhline(color='darkgray', linestyle='--')

    plt.tight_layout()
    fig.savefig(lme_path + 'figures/' + str(dpi) + ' dpi/lme_EffSize_advXroi_' + tw + '_' + str(dpi) + 'dpi.png');
    fig.savefig(lme_path + 'figures/svg/lme_EffSize_advXroi_' + tw + '.svg');
    plt.close()

    # PLOT 4
    ## Point plots of *model estimates* of V3-V2 contrast, arranged by ROI

    fig, axs = plt.subplots(3, 3, figsize=by_roi_figsize)    

    for idx, ax in enumerate(axs.reshape(-1)):
        roi = roi_order[idx]

        ax.axhline(color='darkgray', linestyle='--')

        for x, y, c in zip(df[(df['ROI']==roi)]['Adverb'], 
                           df[df['ROI']==roi]['estimate'],
                           colors
                       ):
            ax.plot(x, y, marker='o', color=c)

        for lower, upper, y, c in zip(df[df['ROI']==roi]['lower.CI'], 
                                      df[df['ROI']==roi]['upper.CI'], 
                                      range(len(df[df['ROI']==roi])),
                                      colors
                                  ):
            ax.plot((y,y), (lower,upper), '-', color=c) 


        ax.set_xlabel('')
        if idx == 0:
            ax.set_ylabel('V3-V2 Model Estimate (µV)')
        elif idx % 3 != 0:
            ax.set_yticks([])
            ax.spines['left'].set_visible(False)

        if idx < 6:
            ax.set_xticks([])    
        else:
            ax.set_xticklabels(adverbs, rotation=0)

        ax.set_title(roi)
        ax.set_ylim(mod_est_ylim)
        ax.tick_params(axis='x', which='both', length=0)

        ax.spines['right'].set_visible(False)
        ax.spines['bottom'].set_visible(False)
        ax.spines['top'].set_visible(False)

    plt.tight_layout()
    fig.savefig(lme_path + 'figures/' + str(dpi) + ' dpi/lme_ModEst_' + tw + '_' + str(dpi) + 'dpi.png');
    fig.savefig(lme_path + 'figures/svg/lme_ModEst_' + tw + '.svg');
    plt.close()

    # PLOT 5
    ## Point plots of each V3-V2 contrast, grouped by adverb (condition)

    fig, axs = plt.subplots(3, 1, figsize=by_adv_figsize)
    for idx, ax in enumerate(axs.reshape(-1)):
        contr = adverbs[idx]

        for label in (ax.get_xticklabels() + ax.get_yticklabels()):
            label.set_fontsize(11)

        ax.plot(df[df['Adverb']==contr]['ROI'], df[df['Adverb']==contr]['estimate'],
                linestyle='None', marker='o', color=color_map[contr])

        for lower, upper, y in zip(df[df['Adverb']==contr]['lower.CI'], 
                                   df[df['Adverb']==contr]['upper.CI'], 
                                   range(len(df))):
            ax.plot((y,y), (lower,upper),'-', color=color_map[contr])

        ax.axhline(0, color='darkgray', linestyle='--')
        ax.axvline(2.5, color='lightgray')
        ax.axvline(5.5, color='lightgray')

        if idx == 0:
            ax.set_ylabel('V3-V2 model estimate (µV)')
        else:
             ax.set_ylabel('')

        if idx == 2:
            ax.set_xlabel('ROI')
            ax.set_xticklabels(roi_order, rotation=0)
        else:
            ax.set_xticks([])

        ax.set_title(contr)

        ax.set_ylim(mod_est_ylim)
        plt.tick_params(
            axis='x',          # changes apply to the x-axis
            which='both',      # both major and minor ticks are affected
            bottom=False,      # ticks along the bottom edge 
            top=False,         # ticks along the top edge 
            labelbottom=True) # labels along the bottom edge

        ax.spines['right'].set_visible(False)
        ax.spines['bottom'].set_visible(False)
        ax.spines['top'].set_visible(False)

    plt.tight_layout()
    fig.savefig(lme_path + 'figures/' + str(dpi) + ' dpi/lme_ModEst_byAdverb' + '_' + tw + '_' + str(dpi) + 'dpi.png');
    fig.savefig(lme_path + 'figures/svg/lme_ModEst_byAdverb' + '_' + tw + '.svg');
    plt.close()
    
    # PLOT 6
    ## Point plots of model estimates, columns are adverbs and rows are ant/mid/cent
    fig, axs = plt.subplots(3, 3, figsize=by_roi_figsize)    

    for idx, ax in enumerate(axs.reshape(-1)):
        adverb = adverbs[idx % 3]
        rois = [r for r in roi_order if roi_levels[idx // 3] in r]   

        ax.plot(rois, 
                df[(df['ROI'].isin(rois)) & (df['Adverb']==adverb)]['estimate'], 
                linestyle='None', marker='o', color=color_map[adverb]
               )

        for lower, upper, y in zip(df[df['ROI'].isin(rois) & (df['Adverb']==adverb)]['lower.CI'], 
                                      df[df['ROI'].isin(rois) & (df['Adverb']==adverb)]['upper.CI'], 
                                      range(len(df[df['ROI'].isin(rois) & (df['Adverb']==adverb)])),

                                  ):
            ax.plot((y,y), (lower,upper), '-', color=color_map[adverb]) 

        if idx < 3:
            ax.set_title(adverb)

        if idx == 0:
            ax.set_ylabel('V3-V2 model estimate (µV)')
        elif idx % 3 != 0:
            ax.set_yticks([])
            ax.spines['left'].set_visible(False)

        ax.set_ylim(mod_est_ylim)

        ax.set_xticklabels(rois, rotation=0)
        ax.tick_params(axis='x', which='both', length=0)

        ax.spines['right'].set_visible(False)
        ax.spines['bottom'].set_visible(False)
        ax.spines['top'].set_visible(False)

        ax.axhline(color='darkgray', linestyle='--')

    plt.tight_layout()
    fig.savefig(lme_path + 'figures/' + str(dpi) + ' dpi/lme_ModEst_advXroi_' + tw + '_' + str(dpi) + 'dpi.png');
    fig.savefig(lme_path + 'figures/svg/lme_ModEst_advXroi_' + tw + '.svg');
    plt.close()


/var/folders/x0/pvpjfdzd39sc5rqvsk5lvv4m0000gn/T/ipykernel_39166/3279600650.py:97: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(roi_order, rotation=0)
/var/folders/x0/pvpjfdzd39sc5rqvsk5lvv4m0000gn/T/ipykernel_39166/3279600650.py:152: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(rois, rotation=0)
/var/folders/x0/pvpjfdzd39sc5rqvsk5lvv4m0000gn/T/ipykernel_39166/3279600650.py:152: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(rois, rotation=0)
/var/folders/x0/pvpjfdzd39sc5rqvsk5lvv4m0000gn/T/ipykernel_39166/3279600650.py:152: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(rois, rotation=0)
/var/folders/x0/pvpjfdzd39sc5rqvsk5lvv4m0000gn/T/ipykernel_39166/3279600650.py:152: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(rois, rotation=0)
/var/f